#### Name: Noha Magdy Ali 
#### ID: 1610126
# Age and Gender Recognition using openCV and CNNs 

### Needed packages 

In [38]:
from skimage import data, io
from matplotlib import pyplot as plt
import numpy as np
from skimage import img_as_ubyte,img_as_float
from skimage import data, io, filters
from matplotlib.pyplot import imshow, show
from skimage.morphology import watershed
from skimage.measure import label
from skimage.color import label2rgb
from skimage import feature
from skimage.filters import threshold_otsu,  threshold_local, sobel
from skimage.morphology import erosion,dilation,opening,closing,square,disk
from skimage.measure import label
from skimage.color import label2rgb
import os
import pandas as pd
import tensorflow as tf
import functools
!pip install mitdeeplearning
import mitdeeplearning as mdl
from tqdm import tqdm
import cv2
import tarfile


### Loading and Preparing Data

In [0]:
# file_id = '0BxYys69jI14kSVdWWllDMWhnN2c'
# request = drive_service.files().get_media(fileId=file_id)
# fh = io.BytesIO()
# downloader = MediaIoBaseDownload(fh, request)
# done = False
# while done is False:
#     status, done = downloader.next_chunk()
#     print ("Download %d%%." % int(status.progress() * 100))

# !wget -P /usr https://drive.google.com/drive/u/0/folders/0BxYys69jI14kSVdWWllDMWhnN2c

# my_tar = tarfile.open("/usr/0BxYys69jI14kSVdWWllDMWhnN2c.tar.gz")
# my_tar.extractall('/usr/Dataset') # specify which folder to extract to
# my_tar.close()




In [51]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path= "/content/drive/My Drive/Dataset/"
data= os.listdir(path)
#print (data)
Data = pd.DataFrame(columns=["Image", "Age", "Gender"])


for idx,i in zip(range(len(data)),data):
    age, gender, race, name=i.split("_")
    img = io.imread(path+i)
    Data.loc[idx] = [ img, age, gender]
    #append(pd.Series(data={"Image":img, "Age":age, "Gender":gender}, name=idx))

In [66]:
Data.shape

(3251, 3)

In [67]:
Data.head()

,Image,Age,Gender
0,"[[[249, 250, 252], [249, 250, 252], [249, 250,...",15,0
1,"[[[200, 200, 202], [200, 200, 202], [199, 199,...",13,1
2,"[[[49, 51, 50], [49, 51, 50], [49, 51, 50], [4...",14,0
3,"[[[169, 181, 169], [173, 185, 173], [172, 184,...",14,0
4,"[[[94, 90, 78], [94, 90, 78], [96, 92, 80], [9...",14,0


In [68]:
rand_idxs = np.random.choice(Data.shape[0], 500, True)
test = Data.loc[rand_idxs]
train = Data
train.drop(rand_idxs)

,Image,Age,Gender
0,"[[[249, 250, 252], [249, 250, 252], [249, 250,...",15,0
1,"[[[200, 200, 202], [200, 200, 202], [199, 199,...",13,1
3,"[[[169, 181, 169], [173, 185, 173], [172, 184,...",14,0
4,"[[[94, 90, 78], [94, 90, 78], [96, 92, 80], [9...",14,0
5,"[[[79, 90, 96], [80, 91, 97], [82, 93, 99], [8...",14,1
...,...,...,...
3246,"[[[18, 20, 19], [18, 20, 19], [18, 20, 19], [1...",89,1
3247,"[[[167, 152, 109], [167, 152, 109], [167, 152,...",89,1
3248,"[[[16, 6, 5], [16, 6, 5], [16, 6, 5], [16, 6, ...",89,1
3249,"[[[113, 52, 0], [113, 52, 0], [113, 52, 0], [1...",90,1


In [69]:
train.head()

,Image,Age,Gender
0,"[[[249, 250, 252], [249, 250, 252], [249, 250,...",15,0
1,"[[[200, 200, 202], [200, 200, 202], [199, 199,...",13,1
2,"[[[49, 51, 50], [49, 51, 50], [49, 51, 50], [4...",14,0
3,"[[[169, 181, 169], [173, 185, 173], [172, 184,...",14,0
4,"[[[94, 90, 78], [94, 90, 78], [96, 92, 80], [9...",14,0


In [70]:
test.head()

,Image,Age,Gender
1865,"[[[238, 233, 230], [238, 233, 230], [238, 233,...",40,0
2657,"[[[111, 96, 93], [111, 96, 93], [111, 96, 93],...",55,0
3164,"[[[82, 100, 114], [82, 100, 114], [82, 100, 11...",84,1
2934,"[[[93, 132, 131], [93, 132, 131], [93, 132, 13...",67,0
1470,"[[[197, 198, 202], [197, 198, 202], [197, 198,...",34,0


In [109]:
def get_batch(x):
  rand_idxs = np.random.choice(train.shape[0], x, True)

  x = np.asarray(train.loc[rand_idxs]["Image"])
  #x=np.expand_dims(x, -1)
  print(x[0].shape)
  print(x[1].shape)

 # x=tf.convert_to_tensor(x)
  y = train.loc[rand_idxs]["Gender"]
  y= tf.convert_to_tensor(y)
  return x,y
n,m =get_batch(2)
#print (n)
#print (m)

(621, 328, 3)
(373, 254, 3)


In [0]:
n_filters = 12 

def classifier(n_outputs=1):
    Conv2D = functools.partial(tf.keras.layers.Conv2D, padding='same', activation='relu')
    BatchNormalization = tf.keras.layers.BatchNormalization
    Flatten = tf.keras.layers.Flatten
    Dense = functools.partial(tf.keras.layers.Dense, activation='relu')

    model = tf.keras.Sequential([
    Conv2D(filters=1*n_filters, kernel_size=5,  strides=2),
    BatchNormalization(),
    
    Conv2D(filters=2*n_filters, kernel_size=5,  strides=2),
    BatchNormalization(),

    Conv2D(filters=4*n_filters, kernel_size=3,  strides=2),
    BatchNormalization(),

    Conv2D(filters=6*n_filters, kernel_size=3,  strides=2),
    BatchNormalization(),

    Flatten(),
    Dense(512),
    Dense(n_outputs, activation=None),
  ])
    return model
My_classifier = classifier()

In [91]:
# Training hyperparameters
batch_size = 32
num_epochs = 2  # keep small to run faster
learning_rate = 5e-4

optimizer = tf.keras.optimizers.Adam(learning_rate) # define our optimizer
loss_history = mdl.util.LossHistory(smoothing_factor=0.99) # to record loss evolution
plotter = mdl.util.PeriodicPlotter(sec=2, scale='semilogy')
if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

@tf.function
def standard_train_step(x, y):
    with tf.GradientTape() as tape:
    # feed the images into the model
        #logits = My_classifier(x)
        print (type(x)) 
  #   # Compute the loss
  #       loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)

  # # Backpropagation
  #   grads = tape.gradient(loss, My_classifier.trainable_variables)
  #   optimizer.apply_gradients(zip(grads, My_classifier.trainable_variables))
  #   return loss
    return 0

# The training loop!
for epoch in range(num_epochs):
    for idx in tqdm(range(train.shape[0]//batch_size)):
        # Grab a batch of training data and propagate through the network
        x, y = get_batch(batch_size)
        loss = standard_train_step(x, y)

        # # Record the loss and plot the evolution of the loss as a function of training
        # loss_history.append(loss.numpy().mean())
        # plotter.plot(loss_history.get())

  0%|          | 0/101 [00:00<?, ?it/s]


ValueError: ignored